In [1]:
import sys
import os
import numpy as np
import pandas as pd
import anndata
from rnasieve.preprocessing import model_from_raw_counts

In [2]:
# load indis ref dictionary
indis_ref = pd.read_pickle("D:/githubs/deconference/data/neuron/real_manu/rnasieve/indis_ref")
individuals = list(indis_ref)
G,K = indis_ref[individuals[0]].shape
indis_ref_array = np.zeros((G,K,len(indis_ref)))
for i in range(len(indis_ref)):
    indis_ref_array[:,:,i] = indis_ref[individuals[i]]

In [3]:
# load celltype_mat dictionary
celltype_mat = pd.read_pickle("D:/githubs/deconference/data/neuron/real_manu/rnasieve/celltype_mat")
DA = celltype_mat['DA']
Epen1 = celltype_mat['Epen1']
Sert = celltype_mat['Sert']
FPP = celltype_mat['FPP']
P_FPP = celltype_mat['P_FPP']
U_Neur = celltype_mat['U_Neur']

In [8]:
G = 12400
K = 6
add_bulk_bias = True
lib_size = 500
n_rep = 100
dirichlet_scales = [5]
n_bulks = [86]
n_refs = [11]
cases = ['null','all_diff']

for n_bulk in n_bulks:
    for aa in dirichlet_scales:
        for case in cases:
            for n_ref in n_refs:
                ref_idx_path = ["D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref", str(n_ref),"_rep",str(n_rep),"_bulk",str(n_bulk),"_dirichlet",str(aa),"_corfdr005_",case,'_ref_idx']
                ref_idx_array = pd.read_pickle("".join(ref_idx_path))
                
                bulk_p_path = ["D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref", str(n_ref),"_rep",str(n_rep),"_bulk",str(n_bulk),"_dirichlet",str(aa),"_corfdr005_",case,'_bulk_p']
                bulk_p_array = pd.read_pickle("".join(bulk_p_path))
                
                bulk_idx_path = ["D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref", str(n_ref),"_rep",str(n_rep),"_bulk",str(n_bulk),"_dirichlet",str(aa),"_corfdr005_",case,'_bulk_idx_mat']
                bulk_idx_array = pd.read_pickle("".join(bulk_idx_path))
                
                p_hat = np.zeros((K,n_bulk,n_rep))
                p_hat_ci_l = np.zeros((K,n_bulk,n_rep))
                p_hat_ci_r = np.zeros((K,n_bulk,n_rep))
                
                for i in range(n_rep):
                    print(i)
                    # generate bulk data
                    # find bulk individual X
                    bulk = np.zeros((G,n_bulk))
                    # bulk_X = np.delete(indis_ref_array,(ref_idx_array[i,]-1).astype(int),axis=2)
                    bulk_X = indis_ref_array[:,:,(bulk_idx_array[i,]-1).astype(int)]
                    for j in range(n_bulk):
                        mu = np.dot(bulk_X[:,:,j],bulk_p_array[:,j,i])
                        if add_bulk_bias:
                            mu = (mu.T * np.random.normal(1,0.1,G)).T
                        mu = mu/np.sum(mu)*lib_size*G
                        bulk[:,j] = np.random.poisson(mu)
                        # formulate ref dict 
                    DAi = DA[:,(ref_idx_array[i,]-1).astype(int)]
                    Epen1i = Epen1[:,(ref_idx_array[i,]-1).astype(int)]
                    Serti = Sert[:,(ref_idx_array[i,]-1).astype(int)]
                    FPPi = FPP[:,(ref_idx_array[i,]-1).astype(int)]
                    P_FPPi = P_FPP[:,(ref_idx_array[i,]-1).astype(int)]
                    U_Neuri = U_Neur[:,(ref_idx_array[i,]-1).astype(int)]
    
                    r1=np.where(np.sum(bulk,axis=1)==0)
                    r2 = np.where(np.sum(DAi,axis=1)==0)
                    r3 = np.where(np.sum(Epen1i,axis=1)==0)
                    r4 = np.where(np.sum(Serti,axis=1)==0)
                    r5 = np.where(np.sum(FPPi,axis=1)==0)
                    r6 = np.where(np.sum(P_FPPi,axis=1)==0)
                    r7 = np.where(np.sum(U_Neuri,axis=1)==0)
    
                    rm_idx = np.concatenate((r1,r2,r3,r4,r5,r6,r7),axis=1)
                    rm_idx = np.unique(rm_idx)

                    bulk = np.delete(bulk,rm_idx,axis = 0)
                    DAi = np.delete(DAi,rm_idx,axis = 0)
                    Epen1i = np.delete(Epen1i,rm_idx,axis = 0)
                    Serti = np.delete(Serti,rm_idx,axis = 0)
                    FPPi = np.delete(FPPi,rm_idx,axis = 0)
                    P_FPPi = np.delete(P_FPPi,rm_idx,axis = 0)
                    U_Neuri = np.delete(U_Neuri,rm_idx,axis = 0)

                    ref_count = {'DA':DAi,'Epen1':Epen1i,'Sert':Serti,'FPP':FPPi,'P_FPP':P_FPPi,'U_Neur':U_Neuri}

                    model1, cleaned_psis1 = model_from_raw_counts(ref_count, bulk)
                    out1 = model1.predict(cleaned_psis1)
                    out1_ci = model1.compute_marginal_confidence_intervals(sig=0.05)

                    ci_l = np.zeros((K,n_bulk))
                    ci_r = np.zeros((K,n_bulk))
                    for nb in range(n_bulk):
                        for k in range(K):
                            ci_l[k,nb] = out1_ci[nb][k][0]
                            ci_r[k,nb] = out1_ci[nb][k][1]
                    p_hat[:,:,i] = np.transpose(np.array(out1))
                    p_hat_ci_l[:,:,i] = ci_l
                    p_hat_ci_r[:,:,i] = ci_r

                    output_path = ['D:/githubs/deconference/output/manuscript/real/rnasieve/add_bulk_bias/neuron_ref', str(n_ref),"_rep",str(n_rep),"_bulk",str(n_bulk),"_dirichlet",str(aa),"_",case]
                    
                    
                    np.save(''.join([''.join(output_path),'_p_hat']), p_hat)
                    np.save(''.join([''.join(output_path),'_p_hat_ci_l']), p_hat_ci_l)
                    np.save(''.join([''.join(output_path),'_p_hat_ci_r']), p_hat_ci_r)

                

# ## dirichlet 5, all_diff
# ref_idx = pd.read_pickle("D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref11_rep100_dirichlet5_corfdr005_all_diff_ref_idx")
# reps_keys = list(ref_idx)
# # transform it to an array
# ref_idx_array = np.zeros((100,11))
# for i in range(100):
#     ref_idx_array[i,:] = ref_idx[reps_keys[i]]

# bulk_p = pd.read_pickle("D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref11_rep100_dirichlet5_corfdr005_all_diff_bulk_p")
# # transform it to an array
# bulk_p_array = np.zeros((6,86,100))
# for i in range(100):
#     bulk_p_array[:,:,i] = bulk_p[reps_keys[i]]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [7]:
''.join([''.join(output_path),'_p_hat'])

'D:/githubs/deconference/output/manuscript/real/rnasieve/add_bulk_bias/neuron_ref11_rep100_bulk86_dirichlet5_null_p_hat'

In [31]:
ref_idx_path = ['D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref', str(11),'_rep',str(100),'_bulk',str(86),'_dirichlet',str(5),'_corfdr005_','null','_ref_idx']
ref_idx_array = pd.read_pickle(''.join(ref_idx_path))

In [34]:
ref_idx_array[[1,4,5],]

array([[49, 11, 46, 59, 44, 92, 48, 18, 27, 26,  6],
       [87, 70, 47, 90, 62, 53, 68,  4, 80, 16, 52],
       [42, 15, 53, 33, 51, 26, 32, 58, 22, 12, 90]])

In [ ]:
#p_hat = np.load('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat.npy')
#p_hat_ci_l = np.load('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat_ci_l.npy')
#p_hat_ci_r = np.load('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat_ci_r.npy')

In [10]:
# lib_size = 500
# #p_hat = np.zeros((K,n_bulk,100))
# #p_hat_ci_l = np.zeros((K,n_bulk,100))
# #p_hat_ci_r = np.zeros((K,n_bulk,100))
# for i in range(75,100):
#     print(i)
#     # generate bulk data
#     # find bulk individual X
#     bulk = np.zeros((G,n_bulk))
#     bulk_X = np.delete(indis_ref_array,(ref_idx_array[i,]-1).astype(int),axis=2)
#     for j in range(n_bulk):
#         mu = np.dot(bulk_X[:,:,j],bulk_p_array[:,j,i])
#         mu = mu/np.sum(mu)*lib_size*G
#         bulk[:,j] = np.random.poisson(mu)
#     # formulate ref dict 
#     DAi = DA[:,(ref_idx_array[i,]-1).astype(int)]
#     Epen1i = Epen1[:,(ref_idx_array[i,]-1).astype(int)]
#     Serti = Sert[:,(ref_idx_array[i,]-1).astype(int)]
#     FPPi = FPP[:,(ref_idx_array[i,]-1).astype(int)]
#     P_FPPi = P_FPP[:,(ref_idx_array[i,]-1).astype(int)]
#     U_Neuri = U_Neur[:,(ref_idx_array[i,]-1).astype(int)]
    
#     r1=np.where(np.sum(bulk,axis=1)==0)
#     r2 = np.where(np.sum(DAi,axis=1)==0)
#     r3 = np.where(np.sum(Epen1i,axis=1)==0)
#     r4 = np.where(np.sum(Serti,axis=1)==0)
#     r5 = np.where(np.sum(FPPi,axis=1)==0)
#     r6 = np.where(np.sum(P_FPPi,axis=1)==0)
#     r7 = np.where(np.sum(U_Neuri,axis=1)==0)
    
#     rm_idx = np.concatenate((r1,r2,r3,r4,r5,r6,r7),axis=1)
#     rm_idx = np.unique(rm_idx)
    
#     bulk = np.delete(bulk,rm_idx,axis = 0)
#     DAi = np.delete(DAi,rm_idx,axis = 0)
#     Epen1i = np.delete(Epen1i,rm_idx,axis = 0)
#     Serti = np.delete(Serti,rm_idx,axis = 0)
#     FPPi = np.delete(FPPi,rm_idx,axis = 0)
#     P_FPPi = np.delete(P_FPPi,rm_idx,axis = 0)
#     U_Neuri = np.delete(U_Neuri,rm_idx,axis = 0)
    
#     ref_count = {'DA':DAi,'Epen1':Epen1i,'Sert':Serti,'FPP':FPPi,'P_FPP':P_FPPi,'U_Neur':U_Neuri}
    
#     model1, cleaned_psis1 = model_from_raw_counts(ref_count, bulk)
#     out1 = model1.predict(cleaned_psis1)
#     out1_ci = model1.compute_marginal_confidence_intervals(sig=0.05)
    
#     ci_l = np.zeros((6,86))
#     ci_r = np.zeros((6,86))
#     for nb in range(86):
#         for k in range(6):
#             ci_l[k,nb] = out1_ci[nb][k][0]
#             ci_r[k,nb] = out1_ci[nb][k][1]
#     p_hat[:,:,i] = np.transpose(np.array(out1))
#     p_hat_ci_l[:,:,i] = ci_l
#     p_hat_ci_r[:,:,i] = ci_r
    
#     np.save('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat', p_hat)
#     np.save('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat_ci_l', p_hat_ci_l)
#     np.save('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat_ci_r', p_hat_ci_r)
    

75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
list(out1.columns)

In [ ]:
out1_ci

In [6]:
p_hat

array([[[0.08333092, 0.02327556, 0.15231851, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.0095163 , 0.59807648, ..., 0.        ,
         0.        , 0.        ],
        [0.18835387, 0.32540028, 0.08412049, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.05514429, 0.11296319, 0.09087754, ..., 0.        ,
         0.        , 0.        ],
        [0.02425701, 0.45915054, 0.32398161, ..., 0.        ,
         0.        , 0.        ],
        [0.13350572, 0.00865634, 0.05374013, ..., 0.        ,
         0.        , 0.        ]],

       [[0.09499044, 0.        , 0.10814195, ..., 0.        ,
         0.        , 0.        ],
        [0.16833467, 0.02476264, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.1796537 , 0.04247569, 0.02337353, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.07042867, 0.14957644, 0.22001026, ..., 0.        ,
         0.        , 0.        ],
        [0.1

In [7]:
p_hat.shape

(6, 86, 100)

In [9]:
p_hat[:,:,74]

array([[9.12667134e-02, 0.00000000e+00, 2.17293519e-01, 2.90663338e-01,
        0.00000000e+00, 3.02095786e-02, 1.95483671e-01, 3.63806423e-01,
        1.20122533e-01, 4.05043695e-01, 3.75288405e-01, 7.62615383e-02,
        5.02898828e-01, 1.11442294e-01, 2.88056873e-01, 4.19229244e-01,
        3.27380802e-01, 3.17886921e-02, 1.75431674e-01, 9.81315573e-02,
        1.69861248e-01, 3.43809705e-03, 1.48860456e-01, 0.00000000e+00,
        1.39843355e-02, 8.91548409e-02, 1.10114210e-01, 3.84816396e-03,
        2.14760094e-01, 0.00000000e+00, 3.11862531e-01, 6.62727054e-02,
        1.93782409e-01, 2.25849405e-01, 0.00000000e+00, 1.71348116e-01,
        2.61123753e-01, 2.76868402e-01, 1.44445058e-01, 6.64486470e-02,
        1.34713352e-01, 1.43535532e-01, 0.00000000e+00, 9.52132913e-02,
        0.00000000e+00, 5.53366622e-01, 2.07756321e-01, 1.14499168e-01,
        7.27271827e-02, 0.00000000e+00, 7.19753402e-02, 9.80110336e-02,
        1.52919893e-01, 6.24940360e-02, 2.58135404e-01, 9.910115